# 6th FEB 2026
The goal of this notebook is to do a simple toy version of the IPP algorithm so I better understand it. 

In [1]:
from dataclasses import dataclass
from typing import List, Optional, Tuple
import math

Before doing the actuall code I'll define a "Pairwase alignment" from mouse to chicken, mouse to human and chicken to mouse. 

The `Alignment`, is a class that basically is self explanatory. I use just a simple class because it very much represents the structure of C++ structs.

In [2]:
@dataclass
class Alignment:
    """Represents an alignment block between two species."""
    reference_start: int
    reference_end: int
    query_start: int
    query_end: int
    query_chromosome: int

    def contains(self, position: int) -> bool:
        # Check if the position falls within the reference range
        return self.reference_start <= position <= self.reference_end

In [3]:
TOY_ALIGNMENTS = {
    # Mouse -> Chicken alignments
    ("mouse", "chicken"): [
        Alignment(100, 200, 150, 250, "chr1"),  # Mouse 100-200 aligns to Chicken 150-250
        Alignment(300, 400, 350, 450, "chr1"),  # Mouse 300-400 aligns to Chicken 350-450
        Alignment(500, 600, 550, 650, "chr1"),  # Mouse 500-600 aligns to Chicken 550-650
    ],
    
    # Mouse -> Human alignments (fewer, simulating larger evolutionary distance)
    ("mouse", "human"): [
        Alignment(200, 300, 180, 280, "chr1"),  # Mouse 200-300 aligns to Human 180-280
        Alignment(600, 700, 620, 720, "chr1"),  # Mouse 600-700 aligns to Human 620-720
    ],
    
    # Chicken -> Human alignments
    ("chicken", "human"): [
        Alignment(200, 300, 200, 300, "chr1"),  # Chicken 200-300 aligns to Human 200-300
        Alignment(400, 500, 400, 500, "chr1"),  # Chicken 400-500 aligns to Human 400-500
        Alignment(600, 700, 600, 700, "chr1"),  # Chicken 600-700 aligns to Human 600-700
    ],
}

We also need a class to represent a given `Coordinate`. 

In [4]:
@dataclass
class Coordinate:
    """Represents a genomic coordinate: chromosome and position."""
    chromosome: str
    position: int
    
    def __str__(self):
        return f"{self.chromosome}:{self.position}"

Then, the goal is that, given an input like mouse `Coordinate("chr1", 250)` we want to know what is the human coordinate. 

To do so we need to follow different steps: 
1. Pick if we do it direct (mouse -> human) or indirect (mouse -> chicken -> human). And in the real example, we may want to use other different species and find the optimal path.
2. Find the anchors, ie. the blocks of mouse and human sequence that we know are aligned. We may find that our coordinate is indeed on an anchor, if not we want to find the upper anchor and the downstream anchor.
3. Do a projection, that is a proportional mapping of coordinates. For example, the anchors in mouse are 10 and 20, and in human are 60 and 100: we may assume that the mouse coordinate 15 is in position 80 of human. It is just an interpolation. (BRRRRR I AM SURE SOMETHING GREEDY HERE CAN BE DONE? WHY ONLY AN INTERPOLATION?)
   a) we compute the reference in the reference space as a proportion.
   b) we put that the query on that proportion.
4. Compute the Score of the projection. That basically, the score decays as the distance from the closest anchors increase.
5. Decide what type of element it is based on this score (that is based on the distance (?))

In [5]:
def find_anchors(
    alignments: List[Alignment],
    position: int
) -> Tuple[Optional[Alignment], 
    Optional[Alignment]]:
    """
    Find the upstream (before) and downstream (after) anchors for a position.
    
    Args:
        alignments: List of alignment blocks, sorted by ref_start
        pos: Position to find anchors for
    
    Returns:
        Tuple of (upstream_anchor, downstream_anchor)
    """

    upstream = None
    downstream = None
    # Note that we just care about the reference.
    for alignment in alignments:
        if alignment.contains(position):
            # Position is directly on an alignment - return it as both anchors
            return (alignment, alignment)
        elif alignment.reference_end < position:
            # This alignment is upstream the position
            if upstream is None or alignment.reference_end > upstream.reference_end:
                upstream = alignment
        elif alignment.reference_end > position:
            # This alignment is downstream the position
            if downstream is None or alignment.reference_end < downstream.reference_end:
                downstream = alignment

    return (upstream, downstream)
    

In [6]:
def calculate_score(
    position: int, 
    up_anchor: Alignment, 
    down_anchor: Alignment, 
    half_life_distance: int = 100 # Decay parameter.
) -> float:
    """
    Calculate projection confidence score based on distance to anchors.
    
    Score decreases exponentially as distance from anchors increases.
    Score = 0.5^(min_distance / half_life_distance)
    
    Args:
        position: Position to project
        up_anchor: Upstream anchor
        down_anchor: Downstream anchor
        half_life_distance: Distance at which score becomes 0.5
    
    Returns:
        Score between 0 and 1
    """
    if up_anchor == down_anchor:
        # Direct alignment - perfect score
        return 1.0
    
    # Calculate minimum distance to either anchor
    dist_to_up = position - up_anchor.reference_end
    dist_to_down = down_anchor.reference_start - position

    # Keep the smaller distance
    min_dist = min(dist_to_up, dist_to_down)
    
    # Exponential decay: score = 0.5^(distance/half_life)
    score = math.pow(0.5, min_dist / half_life_distance)
    return max(0.0, min(1.0, score))  # Clamp between 0 and 1

In [7]:
@dataclass
class ProjectionResult:
    """Result of projecting a coordinate from one species to another."""
    reference_coordinate: Coordinate
    query_coordinate: Coordinate
    score: float  # Confidence score between 0 and 1
    path: List[str]  # Species path taken (e.g., ["mouse", "chicken", "human"])
    anchors: Tuple[Alignment, Alignment]  # Upstream and downstream anchors



def project_direct(
    reference_coordinate: Coordinate,
    reference_species: str, 
    query_species: str, 
) -> Optional[ProjectionResult]:
    """
    Try to project a coordinate directly between two species.
    
    Args:
        ref_species: Reference species name
        qry_species: Query species name
        ref_coord: Coordinate in reference species
    
    Returns:
        ProjectionResult if successful, None otherwise
    """

    # Get aliments between te species 
    # NOTE: in this toy example we know the flow of species
    alignments = TOY_ALIGNMENTS.get((reference_species, query_species))
    if not alignments:
        return None

    # Filter alignments for the correct chromosome
    chromosome_alignments = [a for a in alignments if a.query_chromosome == reference_coordinate.chromosome]
    if not chromosome_alignments:
        return None

    # Sort by position to make the next step simpler
    # We could do a divide and conquere aprouch tho.
    chromosome_alignments.sort(key=lambda a: a.reference_start)

    # Find anchors
    up_anchor, down_anchor = find_anchors(chromosome_alignments, reference_coordinate.position)

    if up_anchor is None or down_anchor is None:
        return None

    # Projection (or a very basic interpolation)

    # If the position is indeed in the anchor both anchors be the same
    if up_anchor == down_anchor:
        offset = reference_coordinate.position - up_anchor.reference_start
        query_position = up_anchor.query_start + offset

    else: 
        # Find the relative position of the reference
        relative_position = (reference_coordinate.position - up_anchor.reference_end) / (down_anchor.reference_start - up_anchor.reference_end)
        # Put it on the query
        query_position = up_anchor.query_end + relative_position * (down_anchor.query_start - up_anchor.query_end)


    score = calculate_score(reference_coordinate.position, up_anchor, down_anchor)

    query_coordinate = Coordinate(reference_coordinate.chromosome, int(query_position))

    return ProjectionResult(
        reference_coordinate = reference_coordinate, 
        query_coordinate = query_coordinate, 
        score =  score, 
        path = [reference_species, query_species],
        anchors = (up_anchor, down_anchor)
    )

In [8]:
def classify_conservation(score: float, dc_threshold: float = 0.9, 
                          ic_threshold: float = 0.7) -> str:
    """
    Classify conservation level based on score.
    
    DC (Direct Conservation): High score, well-conserved
    IC (Indirect Conservation): Medium score, conserved via intermediates
    NC (Not Conserved): Low score, poorly conserved
    """
    if score >= dc_threshold:
        return "DC"
    elif score >= ic_threshold:
        return "IC"
    else:
        return "NC"

Before, moving towards the best path, lets try to see if this code till now owrks well: 

In [9]:
print("EXAMPLE 1: Direct Projection (Mouse -> Human)")
print("-" * 70)
coord1 = Coordinate("chr1", 250)  # Mouse position 250
print(f"Input:  Mouse {coord1}")

result1 = project_direct(coord1, "mouse", "human")
if result1:
    print(f"Output: Human {result1.query_coordinate}")
    print(f"Score:  {result1.score:.3f}")
    print(f"Path:   {' -> '.join(result1.path)}")
    print(f"Classification: {classify_conservation(result1.score)}")
else:
    print("Output: No direct projection found")
print()
    

EXAMPLE 1: Direct Projection (Mouse -> Human)
----------------------------------------------------------------------
Input:  Mouse chr1:250
Output: Human chr1:230
Score:  1.000
Path:   mouse -> human
Classification: DC

